In [3]:
%pip install boto3 pyaudio pywhatkit pyttsx3 twilio speechrecognition opencv-python
%pip install smtplib


[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement smtplib (from versions: none)
ERROR: No matching distribution found for smtplib

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
from os import environ

os.environ['EMAIL_ADDRESS'] = "amanmakhija2205@gmail.com"
os.environ['EMAIL_PASS'] = "******************"
os.environ['ACC_SID'] = "*********************"
os.environ['AUTH_TOKEN'] = "******************"
os.environ['ACCESS_KEY'] = "******************"
os.environ['SECRET_KEY'] = "******************"

In [16]:
import os
import cv2
import boto3
import pyaudio
import smtplib
import pywhatkit
import pyttsx3 as px
from twilio.rest import Client
import speech_recognition as sr

rec = sr.Recognizer()
access_key = os.environ.get('ACCESS_KEY')
secret_key = os.environ.get('SECRET_KEY')

def ec2launch(region):
    myec2 = boto3.client("ec2", region_name=region, aws_access_key_id=access_key, aws_secret_access_key=secret_key)
    print(myec2.run_instances(BlockDeviceMappings=[{"DeviceName":"FirstWebServer1"}],ImageId='ami-0ded8326293d3201b', InstanceType="t2.micro", MaxCount=1, MinCount=1))

def create_s3_bucket(bucket_name, region):
    s3 = boto3.client('s3', region_name=region, aws_access_key_id=access_key, aws_secret_access_key=secret_key)
    s3.create_bucket(Bucket=bucket_name, CreateBucketConfiguration={'LocationConstraint': region})
    print(f"Bucket '{bucket_name}' created successfully in region '{region}'.")

def simple_message(number, message):
    account_sid = os.environ.get('ACC_SID')
    auth_token = os.environ.get('AUTH_TOKEN')
    client = Client(account_sid, auth_token)
    message = client.messages.create(from_='+14326662450', to='+91'+number, body=message)
    print(message.sid)

def sns(subject, message):
    sns = boto3.client("sns", aws_access_key_id=access_key, aws_secret_access_key=secret_key)
    sns.publish(
        Message = message,
        Subject = subject,
        TopicArn = "arn:aws:sns:ap-south-1:418272579003:s3Deletion"
    )

def image_cap():
    cap = cv2.VideoCapture(0)
    status,pic = cap.read()
    cv2.imshow("Image", pic)
    cv2.waitKey()
    cv2.destroyAllWindows()
    cap.release()

def email(to_email, subject, content):
    from email.mime.multipart import MIMEMultipart
    from email.mime.text import MIMEText
    EMAIL_ADDRESS =  os.environ.get('EMAIL_ADDRESS')
    EMAIL_PASSWORD = os.environ.get('EMAIL_PASS')
    with smtplib.SMTP_SSL("smtp.gmail.com", 465) as smtp:
        msg = MIMEMultipart()
        msg['From'] = EMAIL_ADDRESS
        msg['To'] = to_email
        msg['Subject'] = subject
        msg.attach(MIMEText(content, 'plain'))

        server = smtplib.SMTP('smtp.gmail.com', 587)
        server.starttls()
        server.login(EMAIL_ADDRESS, EMAIL_PASSWORD)
        text = msg.as_string()
        server.sendmail(EMAIL_ADDRESS, to_email, text)
        server.quit()

def check_status(text, open_list, not_list):
    isPresent = False
    for t in open_list:
        if t in text:
            isPresent = True
            break

    for t in not_list:
        if t in text:
            isPresent = False
            break

    return isPresent
        
isSpeaking = True
while isSpeaking:
    with sr.Microphone() as mic :
        spk = px.init()
        spk.setProperty('rate',150)
        spk.setProperty('Volume',1.0)
        spk.say("say something dude")
        spk.runAndWait()
        print("speak")
        audio = rec.listen(mic)
        try :
            choice = rec.recognize_google(audio)
            text = choice.lower()
            #text = choice.split(" ")
            print("you might have said : ", text, end = "\n\n")

            open_statements = ["open", "start", "launch", "run", "execute", "begin", "create", "open up", "send", "click"]
            not_statements = ["dont", "don't", "do not", "not", "never", "no", "donot"]
            exit_statements = ["exit", "quit", "close", "stop", "terminate", "end", "finish", "bye", "goodbye", "see you", "bye", "later", "soon"]

            if "chrome" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    os.system("start chrome")
                    spk.say("chrome has been opened.")
                    spk.runAndWait()

            elif "notepad" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    os.system("notepad")
                    spk.say("notepad has been opened.")
                    spk.runAndWait()

            elif "instance" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    ec2launch("ap-south-1")
                    spk.say("ec2 instance has been launched. check on your aws console.")
                    spk.runAndWait()

            elif "whatsapp" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    number = input("Enter the number : ")
                    message = input("Enter the message : ")
                    pywhatkit.sendwhatmsg_instantly("+91"+number, message)
                    spk.say("Message has been sent on whatsapp")
                    spk.runAndWait()

            elif "email" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    to = input("Enter the email address : ")
                    subject = input("Enter the subject : ")
                    content = input("Enter the body of the email : ")
                    email(to, subject, content)
                    spk.say("email has been sent on mail")
                    spk.runAndWait()

            elif "message" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    n = input("Enter the number : ")
                    m = input("Enter message to be sent : ")
                    simple_message(n, m)
                    spk.say("Message has been sent through simple message")
                    spk.runAndWait()

            elif "notification" in text or "sns" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    sub = input("Enter the subject : ")
                    message = input("Enter the message : ")
                    sns(sub, message)
                    spk.say("simple notification message has been sent")
                    spk.runAndWait()
                
            elif "bucket" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    name = input("Enter the name of bucket : ")
                    create_s3_bucket(name, "ap-south-1")
                    spk.say("bucket has been created with name {}".format(name))
                    spk.runAndWait()

            elif "picture" in text or "image" in text :
                if not check_status(text, open_statements, not_statements) :
                    spk.say("okay!")
                    spk.runAndWait()

                else :
                    image_cap()
                    spk.say("picture clicked")
                    spk.runAndWait()

            else :
                if not check_status(text, exit_statements, not_statements) :
                    spk.say("Sorry! I didn't understood what you said.")
                    spk.runAndWait()

                else :
                    isSpeaking = False
                    spk.say("Thank you for using me")
                    spk.runAndWait()
                    break

        except Exception as err :
            print("Error : ", err, end = "\n\n")

SyntaxError: invalid syntax. Maybe you meant '==' or ':=' instead of '='? (3614242591.py, line 17)